<left>FINM 36700 - Portfolio Theory and Risk Management</left>
<br>
<left>Fall 2022</left>

<h2><center> Final Exam </center></h2>

<center>Monday, December 5th 2022</center>

<h3>Ki Hyun</h3>

<h3>CNetID: kwhyun23</h3>

## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from scipy.stats import kurtosis, skew
from scipy.stats import norm

import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.neural_network import MLPRegressor

from arch import arch_model
from arch.univariate import GARCH, EWMAVariance

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

import matplotlib.pyplot as plt

## Helper Functions

In [2]:
import Portfolio_Helper_Functions as function

## Data